In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [2]:
data1=pd.read_csv("C:/Users/visha/OneDrive/Documents/processeddata.csv")
data1

,Unnamed: 0,review,sentiment,processed,word count,length
0,0,One reviewers mentioned watching 1 Oz episode ...,1,one reviewers mentioned watching 1 oz episode ...,215,215
1,1,wonderful little production. <br / ><br / >The...,1,wonderful little production the filming techni...,116,116
2,2,thought wonderful way spend time hot summer we...,1,thought wonderful way spend time hot summer we...,116,116
3,3,Basically ' family little boy ( Jake ) thinks ...,0,basically family little boy jake thinks zo...,92,92
4,4,"Petter Mattei ' "" Love Time Money "" visually s...",1,petter mattei love time money visually stun...,160,160
...,...,...,...,...,...,...
49577,49995,thought movie right good job. ' creative origi...,1,thought movie right good job creative origina...,114,114
49578,49996,"Bad plot , bad dialogue , bad acting , idiotic...",0,bad plot bad dialogue bad acting idiotic di...,73,73
49579,49997,Catholic taught parochial elementary schools n...,0,catholic taught parochial elementary schools n...,142,142
49580,49998,' going disagree previous comment side Maltin ...,0,going disagree previous comment side maltin o...,135,135


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocessing the data for SVM and Logistic Regression models
#tfidf = TfidfVectorizer(max_features=5000)
#X = tfidf.fit_transform(data1['processed']).toarray()
#y = data1['sentiment']

In [4]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train Set')
print(X_train, '\n')
print(X_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
[[0.22946382 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] 

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.07657825 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] 

Test Set
7827     0
4811     0
35252    1
3446     0
24377    0
        ..
11284    0
44732    0
3

In [5]:
def get_max_length():
    review_length = []
    for review in X_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [6]:
# ENCODE SENTIMENT -> 0 & 1
x_data = data1['review']       
y_data = data1['sentiment'] 
y_data = y_data.replace('positive', 1)
y_data = y_data.replace('negative', 0)

In [7]:
from keras.datasets import imdb
max_features = 20000

print('Loading data ...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')


Loading data ...
25000 train sequences
25000 test sequences


In [8]:
print(X_train[0])#Lets take a look on item 0 of this data

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [9]:
from keras.preprocessing import sequence
maxlen = 80
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')
max_length = get_max_length()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('Encoded X Train\n', X_train, '\n')
print('Encoded X Test\n', X_test, '\n')
print('Maximum review length: ', max_length)


X_train shape: (25000, 80)
X_test shape: (25000, 80)
Encoded X Train
 [[   1   14   22 ...    4   22   17]
 [   1  194 1153 ...    9   45   43]
 [   1   14   47 ...  278   36   69]
 ...
 [   1   11    6 ...  960  108   45]
 [   1 1446 7079 ...   40    5  421]
 [   1   17    6 ...  109   29  127]] 

Encoded X Test
 [[   1  591  202 ...    0    0    0]
 [   1   14   22 ...   85  108  131]
 [   1  111  748 ...   31 2122   33]
 ...
 [   1   13 1408 ...    0    0    0]
 [   1   11  119 ...   12    5   55]
 [   1    6   52 ...    9  194   21]] 

Maximum review length:  80


In [10]:
X_train


array([[   1,   14,   22, ...,    4,   22,   17],
       [   1,  194, 1153, ...,    9,   45,   43],
       [   1,   14,   47, ...,  278,   36,   69],
       ...,
       [   1,   11,    6, ...,  960,  108,   45],
       [   1, 1446, 7079, ...,   40,    5,  421],
       [   1,   17,    6, ...,  109,   29,  127]])

In [11]:
for i in range(0,3):
    print (len(X_train[i]))


80
80
80


In [12]:



model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # 128 = dimensionality of the output space
model.add(Dense(1, activation='sigmoid')) 

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)


In [14]:
print('Train...')
model.fit(X_train, y_train,
          batch_size=200,
          epochs=3,
          validation_data=(X_test, y_test),callbacks=[checkpoint])

Train...
Epoch 1/3
125/125 [==============================] - ETA: 0s - loss: 0.5157 - accuracy: 0.7291
Epoch 1: accuracy improved from -inf to 0.72908, saving model to models\LSTM.h5
125/125 [==============================] - 223s 2s/step - loss: 0.5157 - accuracy: 0.7291 - val_loss: 0.4322 - val_accuracy: 0.8000
Epoch 2/3
125/125 [==============================] - ETA: 0s - loss: 0.3066 - accuracy: 0.8772
Epoch 2: accuracy improved from 0.72908 to 0.87716, saving model to models\LSTM.h5
125/125 [==============================] - 100s 804ms/step - loss: 0.3066 - accuracy: 0.8772 - val_loss: 0.4438 - val_accuracy: 0.7937
Epoch 3/3
125/125 [==============================] - ETA: 0s - loss: 0.2182 - accuracy: 0.9184
Epoch 3: accuracy improved from 0.87716 to 0.91840, saving model to models\LSTM.h5
125/125 [==============================] - 109s 871ms/step - loss: 0.2182 - accuracy: 0.9184 - val_loss: 0.5163 - val_accuracy: 0.7844


In [15]:
score, acc = model.evaluate(X_test, y_test, batch_size=200)

print('Test score:', score)
print('Test accuracy:', acc)

125/125 [==============================] - 11s 92ms/step - loss: 0.5163 - accuracy: 0.7844
Test score: 0.516304075717926
Test accuracy: 0.7843999862670898


In [17]:
y_pred = model.predict_classes(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

AttributeError: 'Sequential' object has no attribute 'predict_classes'